In [1]:
import logging.config
import os.path as op

import yaml

from agents.matchmanager import buildMatchManager

if __name__ == '__main__':
    mm = buildMatchManager('', 'scenarioTest1v1', 'PlayerDummy', 'PlayerDummy', seed=42)

    vv=[]
    while not mm.end:
        mm.nextStep()
        vv.append(mm.state.vector())

In [2]:
vv

[(('fid', 'fdf48881-b109-4153-83d1-be87afb374ad'),
  ('team', 'red'),
  ('name', 'Tank1'),
  ('index', 0),
  ('kind', 2),
  ('move', 7),
  ('load', 1),
  ('hp', 1),
  ('hp_max', 1),
  ('int_atk', 6),
  ('int_def', 0),
  ('endurance', 0),
  ('stat_name', 'No effect'),
  ('stat_valuebonus', 0),
  ('activated', 0),
  ('responded', False),
  ('attacked', False),
  ('moved', False),
  ('passed', False),
  ('killed', False),
  ('hit', False),
  ('attacked_by', False),
  ('can_transport', -1),
  ('transport_capacity', True),
  ('len transporting', 2),
  ('transported_by', 0),
  ('positionX', -1),
  ('positionY', 2),
  ('positionZ', -4),
  ('defense basic', 2),
  ('defense smoke', 5),
  ('defense antitank', 18),
  ('weapon CA', 0),
  ('weapon AR', 8),
  ('weapon MG', 0),
  ('weapon AT', 1000000000),
  ('weapon MT', 0),
  ('weapon GR', 0),
  ('weapon SM', 0),
  ('weapon SR', 2),
  ('fid', '7239d7eb-7d6e-4b58-b2ff-5788a4f64502'),
  ('team', 'blue'),
  ('name', 'Tank2'),
  ('index', 0),
  ('kind'

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame.from_items(vv) 

C:\Users\Nicol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  """Entry point for launching an IPython kernel.


ValueError: too many values to unpack (expected 2)

In [ ]:
df

In [ ]:
mm1=buildMatchManager('', 'scenarioTest1v1', 'PlayerDummy', 'PlayerDummy', seed=42)

In [ ]:
mm1.state

In [ ]:
vectors=[]

for i in range(10):
    mm.nextStep()
    s=mm.state
    print(s)
    vectors.append(s.vector())

In [ ]:
vectors

In [ ]:
df=pd.DataFrame(vv)

In [ ]:
df

In [5]:
from agents.matchmanager import buildMatchManager

mm = buildMatchManager('', 'scenarioTest1v1', 'PlayerDummy', 'PlayerDummy', seed=42)


In [6]:
vectors=[]
for i in range(10):
    mm.nextStep()
    s=mm.state
    print(s.lastAction)
    vectors.append(s.vector())

None
RED  : Tank1     : Moved to (5, 7)
RED  : Tank1     : Moved to (5, 7)
BLUE : Tank2     : Moved to (14, 12)
BLUE : Tank2     : Moved to (14, 12)
BLUE : Tank2     : Moved to (14, 12)
RED  : Tank1     : Moved to (6, 6)
RED  : Tank1     : Moved to (6, 6)
BLUE : Tank2     : Moved to (9, 10)
BLUE : Tank2     : Moved to (9, 10)


In [8]:
df= pd.DataFrame(vectors)

In [10]:
from agents.matchmanager import buildMatchManager
mm = buildMatchManager('', 'scenarioTest1v1', 'PlayerDummy', 'PlayerDummy', seed=42)


In [11]:
vectors=[]
for i in range(10):
    mm.nextStep()
    s=mm.state
    print(s.lastAction)
    vectors.append(s.vector())

None
RED  : Tank1     : Moved to (5, 7)
RED  : Tank1     : Moved to (5, 7)
BLUE : Tank2     : Moved to (14, 12)
BLUE : Tank2     : Moved to (14, 12)
BLUE : Tank2     : Moved to (14, 12)
RED  : Tank1     : Moved to (6, 6)
RED  : Tank1     : Moved to (6, 6)
BLUE : Tank2     : Moved to (9, 10)
BLUE : Tank2     : Moved to (9, 10)


In [12]:
df=pd.DataFrame(vectors)

In [17]:
df

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
1,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
2,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
3,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
4,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
5,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
6,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
7,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
8,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"
9,"(fid, 02e395b2-5010-4ca5-94e1-db4525c342a9)","(team, red)","(name, Tank1)","(index, 0)","(kind, 2)","(move, 7)","(load, 1)","(hp, 1)","(hp_max, 1)","(int_atk, 6)",...,"(defense smoke, 5)","(defense antitank, 18)","(weapon CA, 0)","(weapon AR, 8)","(weapon MG, 0)","(weapon AT, 1000000000)","(weapon MT, 0)","(weapon GR, 0)","(weapon SM, 0)","(weapon SR, 2)"


In [13]:
for y in df.iloc[0]:
    print(y)

('fid', '02e395b2-5010-4ca5-94e1-db4525c342a9')
('team', 'red')
('name', 'Tank1')
('index', 0)
('kind', 2)
('move', 7)
('load', 1)
('hp', 1)
('hp_max', 1)
('int_atk', 6)
('int_def', 0)
('endurance', 0)
('stat_name', 'No effect')
('stat_valuebonus', 0)
('activated', 0)
('responded', False)
('attacked', False)
('moved', False)
('passed', False)
('killed', False)
('hit', False)
('attacked_by', False)
('can_transport', -1)
('transport_capacity', True)
('len transporting', 2)
('transported_by', 0)
('positionX', -1)
('positionY', 2)
('positionZ', -4)
('defense basic', 2)
('defense smoke', 5)
('defense antitank', 18)
('weapon CA', 0)
('weapon AR', 8)
('weapon MG', 0)
('weapon AT', 1000000000)
('weapon MT', 0)
('weapon GR', 0)
('weapon SM', 0)
('weapon SR', 2)
('fid', 'a3631ce0-5ca6-4751-a3be-2bf529aabd55')
('team', 'blue')
('name', 'Tank2')
('index', 0)
('kind', 2)
('move', 7)
('load', 1)
('hp', 1)
('hp_max', 1)
('int_atk', 6)
('int_def', 0)
('endurance', 0)
('stat_name', 'No effect')
('stat_